In [ ]:
from NovelSwarmBehavior.novel_swarms.behavior.AngularMomentum import AngularMomentumBehavior
from NovelSwarmBehavior.novel_swarms.behavior.AverageSpeed import AverageSpeedBehavior
from NovelSwarmBehavior.novel_swarms.behavior.GroupRotationBehavior import GroupRotationBehavior
from NovelSwarmBehavior.novel_swarms.behavior.RadialVariance import RadialVarianceBehavior
from NovelSwarmBehavior.novel_swarms.behavior.ScatterBehavior import ScatterBehavior
from NovelSwarmBehavior.novel_swarms.behavior.SensorOffset import GeneElementDifference
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from generation.evolution import ModifiedHaltingEvolution, ModifiedNoveltyArchieve
from generation.halted_evolution import HaltedEvolution
from data.swarmset import SwarmDataset, DataBuilder
import numpy as np
from scipy import ndimage
import cv2
import pygame

def resizeInput(X, w=200):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(w, w), interpolation=cv2.INTER_AREA)
    return resized

def update_network(network, loss_fn, optim, anchor_image, pos_image, neg_image):
    optim.zero_grad()
    anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_image, pos_image, neg_image)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def evolution_controller():
    sensors = SensorSet([
        GenomeBinarySensor(genome_id=8, draw=False),
        GenomeBinarySensor(genome_id=9, draw=False)
    ])

    agent_config = DiffDriveAgentConfig(
        sensors=sensors,
        seed=None,
    )

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=((1/3) * np.pi), _min=-((2/3) * np.pi), mutation_step=(np.pi/2), round_digits=4),
        GeneRule(_max=((2/3) * np.pi), _min=-((1/3) * np.pi), mutation_step=(np.pi/2), round_digits=4),
    ]

    phenotype = [
        AverageSpeedBehavior(),
        AngularMomentumBehavior(),
        RadialVarianceBehavior(),
        ScatterBehavior(),
        GroupRotationBehavior(),
        # GeneElementDifference(8, 9)
    ]

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=24,
        seed=None,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=15,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=15,
        simulation_lifespan=1200,
        display_novelty=True,
        save_archive=True,
        show_gui=True,
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen,
    )

    halted_evolution = ModifiedHaltingEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    return halted_evolution, screen

In [ ]:
import numpy as np
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL
import time
from networks.ensemble import Ensemble

trial_name = f"{str(int(time.time()))}"

TRAIN = True
CLUSTER_AND_DISPLAY = True
WRITE_OUT = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ensemble = Ensemble(size=3, output_size=5, lr=30e-4, learning_decay=0.9, decay_step=4, threshold=9.0, weight_decay=1e-6, new_model=True)
ensemble.load_ensemble("dual-mini-hope")
# ensemble.load_ensemble("full-mini-HIL")
network = ensemble.ensemble[2]
network.eval()

SAVE_CLUSTER_IMAGES = True
SAVE_CLUSTER_MEDOIDS = True
PRETRAINING = True
HUMAN_IN_LOOP = True
SYNTHETIC_HIL = False
EVOLUTION = True
EPOCHS = 20
DATA_SIZE = 10000
EVOLUTIONS = 15
K_CLUSTERS = 8

anchor_dataset = ContinuingDataset("data")
sampled_dataset = SwarmDataset("data/full-mini-dual", rank=0)
evolution, _ = evolution_controller()
dataset = anchor_dataset if EVOLUTION else sampled_dataset
random_archive = DataAggregationArchive(scalar=True)

hil = HIL(name=trial_name, synthetic=SYNTHETIC_HIL, data_limiter=DATA_SIZE, clusters=K_CLUSTERS)
HIL_archive = DataAggregationArchive()

for gen in range(EVOLUTIONS):
    # Simulate current population + Save Data
    for i in range(len(evolution.getPopulation())):
        # The collection of the original behavior vector below is only used to collect data to compare with the baseline
        visual_behavior, genome, baseline_behavior = evolution.next()
        dataset.new_entry(visual_behavior, genome, baseline_behavior)

    medoid_acc, cluster_acc = hil.record_medoids(network, dataset)

    # Then, evolve
    start_time = time.time()
    embedded_archive = hil.getEmbeddedArchive(dataset, network)
    evolution.overwriteArchive(embedded_archive, random_archive)
    embedded_behavior = embedded_archive.archive[-evolution.evolve_config.population:]
    evolution.overwriteBehavior(embedded_behavior)
    evolution.saveArchive(trial_name)
    evolution.evolve()
    evolution.restart_screen()

    print(f"Evolution complete for gen{gen}")

    # Cluster current dataset, display clusters, and save for analysis
    if len(dataset) > 0 and SAVE_CLUSTER_IMAGES:
        hil.embed_and_cluster(network, dataset, auto_quit=True)
        evolution.restart_screen()

In [ ]:
from ui.clustering_gui import ClusteringGUI
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()